In [ ]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry
from api_key import APIKEY

API_KEY = APIKEY

with MPRester(API_KEY) as mpr:
    docs = mpr.materials.summary.search(
        energy_above_hull=(0, 0.008), band_gap=[0.85,2.15], is_stable=True, fields=["material_id"]
    )
    mats = [doc.material_id for doc in docs]

In [ ]:
materials = mats[1500:2000]
bandstruct = []
failed_ids = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        try:
            band_structure = mpr.get_bandstructure_by_material_id(mp_id)
            bandstruct.append(band_structure)
        except Exception as e:
            print(f"Failed to retrieve band structure for {mp_id}: {e}")
            failed_ids.append(mp_id)

# Remove the failed IDs from the materials list
for id in failed_ids:
    materials.remove(id)


In [ ]:
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        material_entries = mpr.materials.search(material_ids=[mp_id])
        
        if material_entries:
            mp_entries.append(material_entries[0])

In [ ]:
print(len(mp_entries),len(bandstruct))

In [ ]:
chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')] 
materials_gga = [material + "-GGA" for material in materials]
materials_id = materials

In [ ]:
bandgaps = []

for i in bandstruct:
    bandgap = i.get_band_gap()
    bandgapenergy = bandgap.get('energy')
    bandgaps.append(bandgapenergy)

In [ ]:
electroneg = []

for i in composition_list:
    electroneg.append(i.average_electroneg)

In [ ]:
cbm = []
vbm = []

for i,x in zip(bandgaps,electroneg):

    bandcorrection = (i + 0.85)/2
    cbm_pre = x + bandcorrection
    vbm_pre = x - bandcorrection
    cbm_minusvac = cbm_pre - 4.5
    vbm_minusvac = vbm_pre - 4.5

    cbm.append(cbm_minusvac)
    vbm.append(vbm_minusvac)

In [ ]:
band_edge = list(zip(cbm, vbm, materials))

In [ ]:
CO_reduction_met = [item for item in band_edge if item[0] >= 0.12]
CO_reduction_below = [item for item in band_edge if 0 <= item[0] < 0.12]


In [ ]:
print("above CO:",len(CO_reduction_met),"||","below CO:",len(CO_reduction_below))

In [ ]:
print("above CO:",CO_reduction_met)
new_list = [item[2] for item in CO_reduction_met]
print(new_list)

In [ ]:
cbms_above = [item[0] for item in CO_reduction_met]
vbms_above = [item[1] for item in CO_reduction_met]
mps_above = [item[2] for item in CO_reduction_met]

cbms_below = [item[0] for item in CO_reduction_below]
vbms_below = [item[1] for item in CO_reduction_below]
mps_below = [item[2] for item in CO_reduction_below]


In [ ]:
import matplotlib.pyplot as plt

water_oxidation = -1.23
co_reduction = 0.12

plt.figure(figsize=(6, 4))

for i in range(len(CO_reduction_met)):
    plt.hlines(cbms_above[i], i - 0.1, i + 0.1, colors='blue', label='CBM' if i == 0 else "")
    plt.hlines(vbms_above[i], i - 0.1, i + 0.1, colors='red', label='VBM' if i == 0 else "")

plt.axhline(y=water_oxidation, color='green', linestyle='--', label='Water Oxidation')
plt.axhline(y=co_reduction, color='green', linestyle='--', label='CO2 Redox')

# Adding labels and title
plt.ylabel('Energy (eV)')
plt.title('Band Gap Alignment: Above CO Reduction')
plt.legend()

# Show the plot
plt.show()


In [ ]:
water_oxidation = -1.23
co_reduction = 0.12

plt.figure(figsize=(6, 4))

for i in range(len(CO_reduction_below)):
    plt.hlines(cbms_below[i], i - 0.1, i + 0.1, colors='blue', label='CBM' if i == 0 else "")
    plt.hlines(vbms_below[i], i - 0.1, i + 0.1, colors='red', label='VBM' if i == 0 else "")

plt.axhline(y=water_oxidation, color='green', linestyle='--', label='Water Oxidation')
plt.axhline(y=co_reduction, color='green', linestyle='--', label='CO2 Redox')

# Adding labels and title
plt.ylabel('Energy (eV)')
plt.title('Band Gap Alignment: Below CO Reduction')
plt.legend()

# Show the plot
plt.show()